In [16]:
from mlflow_utils import setup, test_data, team_train_test_split, atlantic_division
from mlflow_utils.metrics import classification, regression
import mlflow
# import model
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import pandas as pd
import numpy as np
from data import chance_of_victory

setup() # leave blank, if doing covid or win prob then pass 'covid' or 'win'

In [17]:
model_name = "linear" # place name of model here eg xgboost
problem_type = "regression" # place one of [regression, classification, ts] here

In [18]:
data = test_data(data=problem_type) # returns real dataset

In [19]:
print(data)

dataset(data=                       Team  Points               Opponent Location    FG  \
0           Milwaukee Bucks     127          Brooklyn Nets     Home  42.9   
1        Los Angeles Lakers     114  Golden State Warriors     Home  48.8   
2         Charlotte Hornets     123         Indiana Pacers     Home  42.6   
3           Detroit Pistons      88          Chicago Bulls     Home  43.9   
4           New York Knicks     138         Boston Celtics     Home  43.6   
...                     ...     ...                    ...      ...   ...   
2107  Oklahoma City Thunder     108             Miami Heat     Away  29.4   
2108   New Orleans Pelicans     124      San Antonio Spurs     Away  29.6   
2109   Los Angeles Clippers      92              Utah Jazz     Away  37.2   
2110          Chicago Bulls     102           Phoenix Suns     Away  47.0   
2111         Boston Celtics     126       Sacramento Kings     Away  39.3   

        FGA     FG%    3P   3PA    3P%  ...   BPM  VORP  Oppon

In [30]:
# split data
team = 'Boston Celtics'       # only needed if doing regression
if problem_type == 'regression':
    opponents = [t for t in atlantic_division if t != team]
    x_train, x_test, y_train, y_test = team_train_test_split(team=team, opponents=opponents, data=data)
else:
    x_train, x_test, y_train, y_test = train_test_split(data.data.drop([data.target], axis=1), data.data[data.target], test_size=.25)

In [31]:
from sklearn import linear_model

In [40]:
mlflow.autolog(silent=True)
# initialize model
regr = linear_model.LinearRegression()


with mlflow.start_run() as run:
    mlflow.tracking.MlflowClient().set_tag(run_id=run.info.run_id, key=model_name, value=problem_type)
    # train model
    regr.fit(x_train, y_train)
    y_pred = regr.predict(x_test)
    
    # run classification/regression metrics to log results
    metrics = regression(y_test, y_pred)


In [37]:
from sklearn.metrics import mean_squared_error, r2_score

In [38]:
print("Mean square error:" ,mean_squared_error(y_test, y_pred))
print("R2 Score: ", r2_score(y_test, y_pred))

Mean square error: 3.2073700478909026e+25
R2 Score:  -1.3046103492216594e+23


In [39]:
print(metrics)

{'r2': -1.3046103492216594e+23, 'mse': 3.2073700478909026e+25, 'mae': 2067968422691.947, 'mape': 19524775345.622128}
